## Code testing

During the lecture, we learned about **raise** and **assert** statements. Now we're going to see how to use them in research code setting.

In [11]:
# Data handling packages
import numpy as np  
import pandas as pd 
import pynwb  

# Plotting libraries
import matplotlib.pyplot as plt  

# Set the data root according to OS
import platform
from pathlib import Path

# Pandas display settings
pd.set_option('display.max_columns', None)  # Ensures all columns are shown when printing DataFrames

# Inline plotting for Jupyter Notebooks
%matplotlib inline  

Let us consider loading in data from the Visual Behavior Neuropixels dataset we saw in a previous day. Specifically, let us load a single experimental session.

In [14]:
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = Path("/Volumes/Brain2025/")
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = Path("E:/")
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = Path("/data/")
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = Path("/media/$USERNAME/Brain2025/")

# pick a session_id and get session data
example_sessions = [1139846596, 1152811536, 1069461581 ]
this_session = str(example_sessions[0])
this_filename = f'ecephys_session_{this_session}.nwb'
nwb_path = Path('visual-behavior-neuropixels', 'behavior_ecephys_sessions', this_session, this_filename)

# access the session data with pynwb
session = pynwb.NWBHDF5IO(data_root / nwb_path).read()

Let us load in the trial data and metadata about each recorded unit.

In [34]:
trials = session.trials.to_dataframe() 
units_table = session.units.to_dataframe()
electrodes_table = session.electrodes.to_dataframe()
units_table = units_table.join(electrodes_table, on = 'peak_channel_id')

In this metadata, we have important pieces of information such as the amplitude cutoff, inter-spike-interval (ISI) violations ratio, presence ratio, and activity drift of each recorded unit. Generally, we want to filter neurons by these quantities to find "good" neurons. For example, consider defining the following thresholds on these quantities below.

In [37]:
max_amplitude_cutoff = 0.1
max_isi_violations_ratio = 0.5
min_presence_ratio = 0.7

When loading in the trial data for a specific neuron, we can check these quantities with if and print statements to make sure that our criteria are satisfied.

In [38]:
id_no = 0
unit_amplitude_cutoff = units_table.iloc[id_no]['amplitude_cutoff']
unit_isi_violations_ratio = units_table.iloc[id_no]['isi_violations']
unit_presence_ratio = units_table.iloc[id_no]['presence_ratio']

if not unit_amplitude_cutoff <= max_amplitude_cutoff:
    print(f'Unit amplitude cutoff is {unit_amplitude_cutoff}, must be <= {max_amplitude_cutoff}')
if not unit_isi_violations_ratio <= max_isi_violations_ratio:
    print(f'ISI Violations ratio is {unit_isi_violations_ratio}, must be <= {max_isi_violations_ratio}')
if not unit_presence_ratio >= min_presence_ratio:
    print(f'Presence ratio is {unit_presence_ratio}, must be >= {min_presence_ratio}')

Unit amplitude cutoff is 0.5, must be <= 0.1
ISI Violations ratio is 1.24494174659597, must be <= 0.5


Alternatively, we can use assert statements instead.

In [39]:
id_no = 0
unit_amplitude_cutoff = units_table.iloc[id_no]['amplitude_cutoff']
unit_isi_violations_ratio = units_table.iloc[id_no]['isi_violations']
unit_presence_ratio = units_table.iloc[id_no]['presence_ratio']

assert unit_amplitude_cutoff <= max_amplitude_cutoff, f'Unit amplitude cutoff is {unit_amplitude_cutoff}, must be <= {max_amplitude_cutoff}'
assert unit_isi_violations_ratio <= max_isi_violations_ratio, f'ISI Violations ratio is {unit_isi_violations_ratio}, must be <= {max_isi_violations_ratio}'
assert unit_presence_ratio >= min_presence_ratio, f'Presence ratio is {unit_presence_ratio}, must be >= {min_presence_ratio}'

AssertionError: Unit amplitude cutoff is 0.5, must be <= 0.1

See how at least one of these critera were not satisfied and our code threw an AssertionError letting us know! Now what if we carefully filter our units for these quantities?

In [40]:
units_table = units_table[
    (units_table['amplitude_cutoff'] <= 0.1) &
    (units_table['isi_violations'] <= 0.5) &
    (units_table['presence_ratio'] >= 0.7)
]

Now our code should not throw any AssertionErrors!

In [42]:
id_no = 0
unit_amplitude_cutoff = units_table.iloc[id_no]['amplitude_cutoff']
unit_isi_violations_ratio = units_table.iloc[id_no]['isi_violations']
unit_presence_ratio = units_table.iloc[id_no]['presence_ratio']

assert unit_amplitude_cutoff <= max_amplitude_cutoff, f'Unit amplitude cutoff is {unit_amplitude_cutoff}, must be <= {max_amplitude_cutoff}'
assert unit_isi_violations_ratio <= max_isi_violations_ratio, f'ISI Violations ratio is {unit_isi_violations_ratio}, must be <= {max_isi_violations_ratio}'
assert unit_presence_ratio >= min_presence_ratio, f'Presence ratio is {unit_presence_ratio}, must be >= {min_presence_ratio}'